http://walk.northcol.org/haskell/type-classes/

In [38]:
-- 型クラス(type class)とは
-- Num 型クラスの インスタンス（instance）は，具体的な数値型である Int や Double などです．
{-
 型クラスは，オブジェクト指向プログラミングにおける "クラス" と似通った概念ですが，
 レイヤが違う話なので注意が必要です．
 オブジェクト指向プログラミングにおけるクラスはデータ型であり，
 インスタンスはオブジェクトですが，
 Haskell における型クラスはデータ型のひとつ上の概念であり，
 インスタンスがデータ型です． 
-}
-- 型クラス -> 数値型 -> 値(上に行くほど抽象的)
-- Num -> Int, Double -> 1, 2.0

-- 型クラス制約
-- Num aはaはNumのインスタンスであるという制約を表す。この制約をcontextという
sum :: Num a => [a] -> a
sum []     = 0
sum (x:xs) = x + sum xs

print $ sum [1.4, 2, 3]

6.4

In [22]:
-- 型クラスの定義
-- クラスメソッド(class method)。型クラスに定義できる関数のこと
-- class 宣言には，クラスメソッドの型シグネチャ宣言（型定義？）とデフォルト定義（関数の中身）を記述します．
-- なお，デフォルト定義は省略可能です．
-- 組み込み型クラスであるEqは (==), (/=)という2つのクラスメソッドを持つ
-- 2つのメソッドは互いに参照しあっている、
class Eq a where
  (==), (/=) :: a -> a -> Bool

  x == y = not (x /= y)
  x /= y = not (x == y)

Line 4: Use ==
Found:
not (x /= y)
Why not:
x == yLine 5: Use /=
Found:
not (x == y)
Why not:
x /= y

In [39]:
-- インスタンス宣言
-- Eqクラスのインスタンスとして、Point型を宣言する。instance Eq Pointとして表す
data Point = Pt Double Double

instance Eq Point where
  (Pt x y) == (Pt x' y') = x == x' && y == y'

main = do print $ (Pt 1 2) == (Pt 2 3)  -- 出力: False
          print $ (Pt 1 2) /= (Pt 2 3)  -- 出力: True
          print $ (Pt 1 2) == (Pt 1 2)  -- 出力: True

No instance for (Eq Double) arising from a use of `=='
In the first argument of `(&&)', namely x == x'
In the expression: x == x' && y == y'
In an equation for `==': (interactive:Ghci351.Pt x y) == (interactive:Ghci351.Pt x' y') = x == x' && y == y'

In [36]:
-- クラスの継承(inherit)
-- Eq a => Ord a でOrdクラスのインスタンスはEqクラスのインスタンス
-- でなければならないという制約を表す。つまり，Ord クラスのインスタンスは
-- Eq クラスのクラスメソッド (==), (/=) も実装する必要がある。
-- OrdはEqを継承するともいう。
-- Eq クラスはスーパークラス（superclass），Ord クラスはサブクラス（subclass）と呼ばれます．
data Point = Pt Double

instance Eq Point where
  (Pt x) == (Pt x') = x == x'
  
instance Ord Point where
  compare (Pt x) (Pt x')
    | x == x'   = EQ
    | x < x'    = LT
    | otherwise = GT

main = do print $ (Pt 1) == (Pt 2)  -- 出力: False
          print $ (Pt 1) >= (Pt 2)  -- 出力: False
          print $ (Pt 1) <= (Pt 2)  -- 出力: True

No instance for (IHaskellPrelude.Eq interactive:Ghci326.Point) arising from the superclasses of an instance declaration
In the instance declaration for `Ord interactive:Ghci326.Point'

No instance for (Eq Double) arising from a use of `=='
In the expression: x == x'
In a stmt of a pattern guard for
               an equation for `compare':
  x == x'
In an equation for `compare':
    compare (interactive:Ghci326.Pt x) (interactive:Ghci326.Pt x')
      | x == x' = EQ
      | x < x' = LT
      | otherwise = GT

In [ ]:
-- deriving 宣言
data Point = Pt Double Double deriving Eq

main = do print $ (Pt 1 2) == (Pt 1 2)  -- 出力: True
          print $ (Pt 1 2) == (Pt 2 3)  -- 出力: False

In [46]:
-- 標準の型クラス
-- Eq(== /=のメソッドをもつ) Ord(min, max等の順序付け可能？なメソッドをもつ) 
-- Num 数値として扱われる型を表す。数値が持つメソッドをもつ
data Day = Sun | Mon | Tue | Wed | Thu | Fri | Sat
           deriving (Enum, Show)
main = print [Mon .. Fri]  -- 出力: [Mon, Tue, Wed, Thu, Fri]
main

[Mon,Tue,Wed,Thu,Fri]

In [49]:
-- カインド(kind) 型の型
-- 関数 cmap の型宣言を吟味すると，Container クラスのインスタンスは Maybe であって，Maybe a ではないことがわかります． Container クラスのインスタンスは，具体型を取り具体型を返す，一種の高階の型であると言えます．
-- Container クラスのインスタンスのカインドは，* -> * と書かれます． ここで，カインド * は何らかの具体的な型を表します． カインド * -> * は，具体的な型を取り，具体的な型を返す型を表します． Maybe :: * -> * であり，Int :: * なので，Maybe Int のカインドは * です．

class Container c where
  cmap :: (a -> b) -> c a -> c b
  
-- ContainerのインタンスはMaybe型
instance Container Maybe where
  -- cmap :: (a -> b) -> Maybe a -> Maybe b
  cmap f Nothing  = Nothing
  cmap f (Just x) = Just (f x)

main = do print $ cmap (2*) Nothing   -- 出力: Nothing
          print $ cmap (2*) (Just 3)  -- 出力: Just 6
main

Nothing
Just 6

http://walk.northcol.org/haskell/modules/

In [51]:
-- モジュール(module)、関数や型をパッケージ化する役割を持つ、他の言語においてパッケージ，ストラクチャ，名前空間などと呼ばれている概念と同様です
-- 標準モジュールの利用
import Data.List

main = do print $ sort [2, 1, 3]
          print $ Data.List.sort [2, 1, 3]
main

-- 一部の関数をimportしたくない場合は
import Prelude hiding (take, length)

[1,2,3]
[1,2,3]

In [63]:
-- モジュールの作成
-- Geometry module
module Geometry where
-- エクスポートする属性を指定する。エクスポートリストという
-- module Geometry(Shape(Tri, Rect), area) where

data Shape = Tri { base, height:: Double }
           | Rect { width, height :: Double }

area :: Shape -> Double
area (Tri x y)  = x * y / 2
area (Rect x y) = x * y

.//Geometry.hs: commitBuffer: invalid argument (invalid character)

In [62]:
import Geometry

main = do print $ width (Rect 3.0 4.0)   -- 出力: 3.0
          print $ height (Rect 3.0 4.0)  -- 出力: 4.0
          print $ area (Tri 3.0 4.0)     -- 出力: 6.0
          print $ area (Rect 3.0 4.0)    -- 出力: 12.0

Ambiguous occurrence `width'
It could refer to either `Geometry.width', imported from `Geometry' (and originally defined at Geometry.hs:4:21-25)
                      or `IHaskell.Display.StaticCanvas.width', imported from `IHaskell.Display.StaticCanvas'


Ambiguous occurrence `height'
It could refer to either `Geometry.height', imported from `Geometry' (and originally defined at Geometry.hs:3:26-31)
                      or `IHaskell.Display.StaticCanvas.height', imported from `IHaskell.Display.StaticCanvas'

In [64]:
-- 階層化されたモジュール
-- Dataディレクトリ下にGeometry.hsを入れて、Geometry->Data.Geometryに書き直す必要がある
-- module Data.Geometry

In [65]:
-- Mainモジュール　モジュール名がかあかれていない場合はMainとして扱われる
-- 以下のように書かれる
module Main(main) where
    main = prtStrLn "hello, world"

.//Main.hs: commitBuffer: invalid argument (invalid character)